In [ ]:
import os
import argparse

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Define the required scopes
SCOPES = ["https://www.googleapis.com/auth/tasks.readonly"]

def authenticate():
    """Authenticate user and return Google Tasks service object."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)

        # Save credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("tasks", "v1", credentials=creds)
        return service
    except HttpError as err:
        print(f"An error occurred: {err}")
        return None

def list_task_lists(service, max_results=10):
    """List task lists from the user's Google Tasks account."""
    try:
        results = service.tasklists().list(maxResults=max_results).execute()
        items = results.get("items", [])

        if not items:
            print("No task lists found.")
            return

        print("Task lists:")
        for item in items:
            print(f"- {item['title']} ({item['id']})")

    except HttpError as err:
        print(f"API error: {err}")




service = authenticate()
list_task_lists(service)





FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'